In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
from mpl_toolkits.mplot3d import Axes3D
import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os 
import warnings
warnings.filterwarnings('ignore')

os.environ['CUDA_VISIBLE_DEVICES'] = "0" 
np.random.seed(0)
tf.random.set_random_seed(0)

In [ ]:

class PhysicsInformedNN:
    it = 0
    
    def __init__(self, X_u, u, X_f, layers,lb, ub,vf,rho_j,dropout_rate=0.05):
        self.vf = vf
        self.rho_j = rho_j
        self.lb = lb
        self.ub = ub
    
        self.x_u = X_u[:,0:1]
        self.t_u = X_u[:,1:2]
        
        self.x_f = X_f[:,0:1]
        self.t_f = X_f[:,1:2]
        
        self.v = u[:,0:1]
        self.rho = u[:,1:2]
        
        self.layers = layers
        self.dropout_rate = dropout_rate
        
        self.weights, self.biases = self.initialize_NN(layers)
        
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        self.x_u_tf = tf.placeholder(tf.float32, shape=[None, self.x_u.shape[1]])
        self.t_u_tf = tf.placeholder(tf.float32, shape=[None, self.t_u.shape[1]])        
        self.v_tf = tf.placeholder(tf.float32, shape=[None, self.v.shape[1]])
        self.rho_tf = tf.placeholder(tf.float32, shape=[None, self.rho.shape[1]])
        self.x_f_tf = tf.placeholder(tf.float32, shape=[None, self.x_f.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float32, shape=[None, self.t_f.shape[1]])   
        
        self.keep_prob = tf.placeholder_with_default(0.95, shape=())
                
        self.v_pred, self.rho_pred = self.net_u(self.x_u_tf, self.t_u_tf) 
        self.f_pde, self.f_fd, self.f_loss = self.net_f(self.x_f_tf, self.t_f_tf) 
        
        self.loss = 0.4 * tf.reduce_mean(tf.square(self.v_tf - self.v_pred)) + 0.6 * tf.reduce_mean(tf.square(self.rho_tf - self.rho_pred)) +\
                         0.5* tf.reduce_mean(tf.square(self.f_pde)) + 0.5 *  tf.reduce_mean(tf.square(self.f_fd)) 
                         
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})
        
        self.optimizer_Adam = tf.train.AdamOptimizer(0.001)
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):  
        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0, num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)
        return weights, biases
    
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0         # scaling
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
            H = tf.nn.dropout(H, keep_prob=self.keep_prob)  #  dropout
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
            
    def net_u(self, x, t):
        u = self.neural_net(tf.concat([x,t],1), self.weights, self.biases)
        v = u[:,0:1]
        rho = u[:,1:2]
        return v,rho
    
    def net_f(self, x,t):
        v,rho = self.net_u(x,t)
        rho_t = tf.gradients(rho, t)[0] 
        rho_x = tf.gradients(rho, x)[0]
        v_x = tf.gradients(v, x)[0]
        f_pde = rho_t + rho * v_x + v * rho_x       #LWR的loss
        f_fd = self.vf * tf.exp(-rho/self.rho_j)-v  #基本图loss
        fd = self.vf * tf.exp(-rho/self.rho_j)     
        f_pde_2 = v + vf- fd     
        f_pde_3 = fd - v
        f_pde_2_t = tf.gradients(f_pde_2, t)[0]    
        f_pde_2_x = tf.gradients(f_pde_2, x)[0] 
        f_loss = f_pde_2_t + v * f_pde_2_x - (f_pde_3)/30   #二阶ARZ
        return f_pde, f_fd, f_loss

    def callback(self, loss):
        print('Loss:', loss)
        
    def train(self,nIter):     
        
        tf_dict = {self.x_u_tf: self.x_u, self.t_u_tf: self.t_u, self.v_tf: self.v,self.rho_tf: self.rho,
                   self.x_f_tf: self.x_f, self.t_f_tf: self.t_f, self.keep_prob: 1 - self.dropout_rate}   
        
        start_time = time.time()
        MSE_history=[]
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            if it %100 == 0:
                loss_value = self.sess.run(self.loss, tf_dict)
                print('It: %d, Loss: %.3e' % 
                      (it, loss_value))
                MSE_history.append(loss_value)
        self.optimizer.minimize(self.sess, 
                                feed_dict = tf_dict,         
                                fetches = [self.loss], 
                                loss_callback = self.callback)        
        return MSE_history
    
    def predict(self, X_star):      
            
        v_star = self.sess.run(self.v_pred, {self.x_u_tf: X_star[:,0:1], self.t_u_tf: X_star[:,1:2],self.keep_prob: 1 - self.dropout_rate})  
        rho_star = self.sess.run(self.rho_pred, {self.x_u_tf: X_star[:,0:1], self.t_u_tf: X_star[:,1:2],self.keep_prob: 1 - self.dropout_rate})  
                
        f_v_star = self.sess.run(self.f_pde, {self.x_f_tf: X_star[:,0:1], self.t_f_tf: X_star[:,1:2]})
        f_rho_star = self.sess.run(self.f_fd, {self.x_f_tf: X_star[:,0:1], self.t_f_tf: X_star[:,1:2]})
        
        return v_star, rho_star
    
    
    def predict_mc(self, X_star, n_samples=50):
       
        v_samples = []
        rho_samples = []
        
        print('n_samples',n_samples)

        for _ in range(n_samples):
            tf_dict = {
                self.x_u_tf: X_star[:, 0:1],
                self.t_u_tf: X_star[:, 1:2],
                self.keep_prob: 1 - self.dropout_rate  
            }
            v_pred = self.sess.run(self.v_pred, tf_dict)
            print('---------------------------',v_pred)
            rho_pred = self.sess.run(self.rho_pred, tf_dict)
            v_samples.append(v_pred)
            rho_samples.append(rho_pred)

        v_samples = np.stack(v_samples, axis=0)
        rho_samples = np.stack(rho_samples, axis=0)
        

        v_mean = np.mean(v_samples, axis=0)
        rho_mean = np.mean(rho_samples, axis=0)
        v_std = np.std(v_samples, axis=0)
        rho_std = np.std(rho_samples, axis=0)

        return v_mean, rho_mean, v_std, rho_std,v_samples,rho_samples


In [ ]:
if __name__ == "__main__": 
    layers = [2, 64, 64, 64, 64, 64, 64, 64, 64, 64, 2] #######
    
    import pickle
    file = open('US101_Lane1to5_t1.5s30.pickle', 'rb')
    info = pickle.load(file)
    speed=info['vMat'].T
    density = info['rhoMat'].T
    flat_arrayv = speed.flatten()
    flat_arrayrho = density.flatten()
    
    t=info['t']
    x=info['s']
    X, T = np.meshgrid(x,t)
    df = pd.DataFrame({'speed': flat_arrayv, 'density': flat_arrayrho,'x':x * 1770})
    # ------------- alpha = 0.01 --------------
    # vf = 26.5932
    # rho_j = 0.3008
    # ---------------------------
    # ------------- alpha = 0.08 --------------
    # vf = 25.1762
    # rho_j = 0.2663
    # ---------------------------
     # ------------- alpha = 0.15 --------------
    # vf = 24.8948
    # rho_j = 0.2491
    # ---------------------------
     # ------------- alpha = 0.22 --------------
    # vf = 24.6698
    # rho_j = 0.2386
    # ---------------------------
    # ------------- alpha = 0.29 --------------
    # vf = 24.4427
    # rho_j = 0.2314
    # ---------------------------
    # ------------- alpha = 0.36 --------------
    # vf = 24.2222
    # rho_j = 0.2256
    # ---------------------------
    # ------------- alpha = 0.43 --------------
    # vf = 24.0017
    # rho_j = 0.2207
    # ---------------------------
    # ------------- alpha = 0.50 --------------
    vf = 23.7619
    rho_j = 0.2164
    # ---------------------------
    # # ------------- alpha = 0.57 --------------
    # vf = 23.4925
    # rho_j = 0.2125
    # # ---------------------------
    # ------------- alpha = 0.64 --------------
    # vf = 23.1839
    # rho_j = 0.2089
    # # ---------------------------
    # ------------- alpha = 0.71 --------------
    # vf = 22.8114
    # rho_j = 0.2053
    # # ---------------------------
    # ------------- alpha = 0.78 --------------
    # vf = 22.3361
    # rho_j = 0.2018
    # # ---------------------------
    # ------------- alpha = 0.85 --------------
    # vf = 21.7062
    # rho_j = 0.1977
    # # ---------------------------
    # ------------- alpha = 0.92 --------------
    # vf = 20.6696
    # rho_j = 0.1926
    # # ---------------------------
    # ------------- alpha = 0.99 --------------
    # vf = 15.7383
    # rho_j = 0.1892
    # # ---------------------------
    
    
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = df.loc[:,['speed','density']]
    
    u_star = np.array(u_star.iloc[:,:])              
    
    
    lb = X_star.min(0)
    ub = X_star.max(0)  
    

    m = 12
    if m == 12:
        loopidx = [0,1,2,4,6,8,10,12,14,16,18,20]
    else:
        gap = int(len(x)/(m-1))
        loopidx = [i*gap for i in range(m-1)]
        loopidx.append(len(x)-1)
    span = np.array(loopidx)
    X_u_train = np.hstack([np.meshgrid(np.array(x)[span],t)[0].reshape(-1,1), np.meshgrid(np.array(x)[span],t)[1].reshape(-1,1) ])
    u_train = df[df['x'].isin(df.loc[span,:]['x'])].loc[:,['speed','density']]
    u_train = np.array(u_train.iloc[:,:])
    idx2 = np.random.choice(X_star.shape[0], int(X_star.shape[0] * 0.6), replace=False)
    X_f_train = X_star[idx2,:]
    data_list = [[] for i in range(2)]
    X_u_train = X_u_train[:, :]
    u_train = u_train[:,:]

    model = PhysicsInformedNN(X_u_train, u_train, X_f_train, layers,lb, ub,vf,rho_j,dropout_rate=0.05)
    start_time = time.time()                
    model.train(20000)
    elapsed = time.time() - start_time                
    print('Training time: %.4f' % (elapsed))

    v_pred, rho_pred = model.predict(X_star)
    print(v_pred)
    print(flat_arrayv)


    v_mean, rho_mean, v_std, rho_std,v_samples,rho_samples = model.predict_mc(X_star, n_samples=50)
    print("速度预测均值:", v_mean.flatten()[:5])
    print("速度预测标准差:", v_std.flatten()[:5])

In [ ]:
v_pred = griddata(X_star, v_pred.flatten(), (X, T), method='cubic')
rho_pred = griddata(X_star, rho_pred.flatten(), (X, T), method='cubic')
# path= r'/root/autodl-tmp/code_planA/PIDL_LWR_10loops'
# import os
# np.save(os.path.join(path,'dropout_'+str(0.05)+'v_alpha'+ str(0.50) + '_lwr_loop'+str(m) + '.npy'),arr=v_pred)#######
# np.save(os.path.join(path,'dropout_'+str(0.05)+'rho_alpha'+ str(0.50) + '_lwr_loop'+str(m) + '.npy'),arr=rho_pred)#######
# np.save(os.path.join(path,'dropout_'+str(0.05)+'meanv_alpha'+ str(0.50) + '_lwr_loop'+str(m) + '.npy'),arr=v_mean)#######
# np.save(os.path.join(path,'dropout_'+str(0.05)+'meanrho_alpha'+ str(0.50) + '_lwr_loop'+str(m) + '.npy'),arr=rho_mean)#######
# np.save(os.path.join(path,'dropout_'+str(0.05)+'stdv_alpha'+ str(0.50) + '_lwr_loop'+str(m) + '.npy'),arr=v_std)#######
# np.save(os.path.join(path,'dropout_'+str(0.05)+'stdrho_alpha'+ str(0.50) + '_lwr_loop'+str(m) + '.npy'),arr=rho_std)#######
# np.save(os.path.join(path,'sample50_dropout_'+str(0.05)+'v_alpha'+ str(0.50) + '_lwr_loop'+str(m) + '.npy'),arr=v_samples)#######
# np.save(os.path.join(path,'sample50_dropout_'+str(0.05)+'rho_alpha'+ str(0.50) + '_lwr_loop'+str(m) + '.npy'),arr=rho_samples)#######